In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)

price_gap = 1.1
price_range = 180, 250
# neighs = ['一梯两户','一梯三户','一梯四户']
# neighs = ['一梯两户','一梯三户']
# neighs = ['一梯两户']
neighs = ['一梯三户']
floor = ['中']

In [2]:
data = pd.read_json('./data_3_5/putuo.json')

In [3]:
data_clean = data[['标题', '总价', '每平方售价', '建筑面积', '产权年限', '建造时间', '小区名称', '梯户比例', '所在楼层', '房屋户型', 'url']].rename(index=str, columns={'标题': 'title', '总价': 'total_price', '每平方售价': 'price', '产权年限' : 'period', '建造时间': 'time', '小区名称': 'community', '梯户比例': 'neighbours', '所在楼层': 'floor', '建筑面积':'area', '房屋户型': 'type'})

In [4]:
data_clean.total_price = data_clean.total_price.map(lambda x: x.strip('万')).astype(float)
data_clean.price = data_clean.price.map(lambda x: x.strip('元/平米')).astype(float)
data_clean = data_clean[(data_clean['total_price']<price_range[1]) & (data_clean['total_price']>price_range[0])]

In [5]:
print('总的房源数目: {}'.format(len(data_clean)))

总的房源数目: 553


In [6]:
data_clean.head()

,title,total_price,price,area,period,time,community,neighbours,floor,type,url
1,新沪小区 出门是沪太公园 两年前婚房装修 诚意出售,188.0,43977.0,42.75㎡,未知,1984年建/板楼,新沪小区,一梯四户,高楼层 (共6层),2室0厅1厨1卫,https://sh.lianjia.com/ershoufang/107100656703.html
3,"精装修全明户型,小区中间,随时看房,近地铁7号线",185.0,51807.0,35.71㎡,未知,1986年建/板楼,管弄四街坊,一梯五户,中楼层 (共6层),1室1厅1厨1卫,https://sh.lianjia.com/ershoufang/107100550675.html
5,楼层好，3-4层楼层，朝南一室户，毛坯房，婚房合适,185.0,47497.0,38.95㎡,未知,1985年建/板楼,延长小区(普陀),一梯六户,中楼层 (共6层),1室1厅1厨1卫,https://sh.lianjia.com/ershoufang/107001422153.html
7,内环内 可改两房 3.4号线中谭路站 精装修 单价低,195.0,43683.0,44.64㎡,70年,未知年建/板楼,龚家宅,一梯六户,中楼层 (共7层),1室1厅1厨1卫,https://sh.lianjia.com/ershoufang/107100720139.html
9,婚房装修 品牌家电 基本可以拎包入住 看房有钥匙,193.0,48603.0,39.71㎡,未知,1966年建/板楼,宜川四村,一梯八户,中楼层 (共5层),1室1厅1厨1卫,https://sh.lianjia.com/ershoufang/107100522249.html


In [7]:
print('总的房源数目: {}, 均价: {}'.format(len(data_clean), data_clean.price.mean()))

总的房源数目: 553, 均价: 47616.226039783


In [8]:
community_price_avg = data_clean.groupby('community', as_index=False).price.agg('mean').rename(columns={'price': 'community_avg_price'})
community_price_avg.head()

,community,community_avg_price
0,三源路162弄,65390.0
1,中天SOHO,40047.0
2,中山北路2165弄,54514.5
3,中山北路2185弄,57619.5
4,中山北路2626弄,55599.0


In [9]:
print('总的小区数目: {}'.format(len(community_price_avg)))

总的小区数目: 185


In [10]:
data_clean = data_clean.merge(community_price_avg, how='left', on='community')
data_clean['floor_simple'] = data_clean['floor'].map(lambda x: x[0])

In [11]:
# data_clean.head()

In [12]:
price_mask = (data_clean['price'] < data_clean['community_avg_price'] * price_gap)
floor_mask = (data_clean['floor_simple'].isin(floor))
if len(neighs) == 0:
    fruit = data_clean[price_mask & floor_mask]
else:
    neigh_mask = (data_clean['neighbours'].isin(neighs))
    fruit = data_clean[neigh_mask & price_mask & floor_mask]

In [13]:
print('total: {}, left: {}'.format(len(data_clean), len(fruit)))
fruit

total: 553, left: 11


,title,total_price,price,area,period,time,community,neighbours,floor,type,url,community_avg_price,floor_simple
39,此房一手买进，原始户型，南北通 楼层佳，交通方便,198.0,48459.0,40.86㎡,未知,1993年建/板楼,宜川三村,一梯三户,中楼层 (共6层),1室1厅1厨1卫,https://sh.lianjia.com/ershoufang/107101025499.html,44962.333333,中
144,满五唯一 南北通 采光好无遮挡 近菜市场 配套成熟,245.0,46192.0,53.04㎡,未知,1987年建/板楼,洛川路16号,一梯三户,中楼层 (共7层),2室1厅1厨1卫,https://sh.lianjia.com/ershoufang/107100620435.html,46192.000000,中
168,内环内 小户型 低总价 近中山公园龙之梦月星环球港,240.0,51010.0,47.05㎡,未知,1995年建/板楼,中山北路3904弄,一梯三户,中楼层 (共7层),1室1厅1厨1卫,https://sh.lianjia.com/ershoufang/107100983126.html,50907.500000,中
176,桃浦四村，2室1厅，满五唯一，楼层佳采光好，有钥匙,240.0,37796.0,63.5㎡,未知,1993年建/板楼,桃浦四村易居紫荆苑,一梯三户,中楼层 (共6层),2室1厅1厨1卫,https://sh.lianjia.com/ershoufang/107002332327.html,37938.750000,中
214,地铁沿线3-4-11-13号线，新装修，拎包可住，看房方便,239.0,66537.0,35.92㎡,未知,1990年建/板楼,白玉新村,一梯三户,中楼层 (共6层),1室1厅1厨1卫,https://sh.lianjia.com/ershoufang/107100589493.html,61664.000000,中
225,楼层好 交通便利 三轨交汇 铁路市场 超市 业主诚售！,215.0,58187.0,36.95㎡,未知,1988年建/板楼,曹杨七村,一梯三户,中楼层 (共6层),1室1厅1厨1卫,https://sh.lianjia.com/ershoufang/107100858349.html,58187.000000,中
270,宜川六村 1室1厅 208万,208.0,43226.0,48.12㎡,70年,1995年建/板楼,宜川六村,一梯三户,中楼层 (共4层),1室1厅1厨1卫,https://sh.lianjia.com/ershoufang/107002341831.html,45635.200000,中
336,地铁口，繁华地段，配套成熟，户型方正，出行方便,230.0,39540.0,58.17㎡,未知,1991年建/板楼,桃浦三村和乐苑,一梯三户,中楼层 (共7层),2室1厅1厨1卫,https://sh.lianjia.com/ershoufang/107100901552.html,38201.000000,中
345,南北通一房 精装修 拎包入住 中间楼层,225.0,58427.0,38.51㎡,未知,1984年建/板楼,石泉小区,一梯三户,中楼层 (共5层),1室1厅1厨1卫,https://sh.lianjia.com/ershoufang/107100580080.html,58427.000000,中
487,全明边套，配套齐全，出行方便，人文素质高，适合居住,225.0,58793.0,38.27㎡,未知,1982年建/板楼,童家桥小区,一梯三户,中楼层 (共7层),1室1厅1厨1卫,https://sh.lianjia.com/ershoufang/107100069964.html,58793.000000,中
